In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import controlflow as cf
from pydantic import BaseModel
from typing import List, Optional

- The following code demonstrates a code review system using controlflow.
- It defines a CodeReview model with status, issues, suggestions and approval fields.
- Two tools are created - approve_code() and reject_code() to handle review outcomes.
- A specialized code reviewer agent is created with GPT-4 to analyze code quality.
- The review_code() flow function orchestrates the review process using these components.

In [4]:
# Define our data models
class CodeReview(BaseModel):
    status: str
    issues: List[str]
    suggestions: List[str]
    approval: bool

# Define custom tools
def approve_code(result: CodeReview):
    return CodeReview(
        status="approved",
        issues=[],
        suggestions=["Code meets all quality standards"],
        approval=True
    )

def reject_code(issues: List[str]):
    return CodeReview(
        status="rejected",
        issues=issues,
        suggestions=[],
        approval=False
    )

# Create specialized agent
reviewer = cf.Agent(
    name="Code Reviewer",
    model="openai/gpt-4",
    instructions="""You are an expert code reviewer. 
    Analyze code for issues and provide constructive feedback.
    Only approve code that meets all quality standards.""",
    tools=[approve_code, reject_code]
)

@cf.flow
def review_code(code: str):
    # Create a task with specific completion tools
    result = cf.run(
        objective="Review the provided code and either approve or reject it",
        result_type=CodeReview,
        agents=[reviewer],
        context={
            "code": code,
            "requirements": [
                "Check for proper variable naming",
                "Verify code efficiency",
                "Look for potential bugs",
                "Ensure code readability"
            ]
        }
    )
    
    return result

# Example usage
code_to_review = """
def calculate_sum(numbers):
    sum = 0  # Using reserved keyword as variable name
    for i in range(len(numbers)):  # Could use direct iteration
        sum += numbers[i]
    return sum
"""

try:
    # Run the code review
    result = review_code(code_to_review)
    
    # Print results based on success/failure
    if result.approval:
        print("✅ Code Review Passed")
        print("\nSuggestions:")
        for suggestion in result.suggestions:
            print(f"- {suggestion}")
    else:
        print("❌ Code Review Failed")
        print("\nIssues Found:")
        for issue in result.issues:
            print(f"- {issue}")
        
except Exception as e:
    print(f"Review process failed: {str(e)}")

Output()

❌ Code Review Failed

Issues Found:
- Overwriting built-in function name
- Inefficient list iteration


## Result Validators

Use a result validator to ensure task outputs meet specific criteria, such as data integrity, business rules, or quality standards.


In [5]:
# Define a simple data model for the task result
class NumberResult(BaseModel):
    number: int

# Define a result validator function
def validate_even(result: NumberResult) -> NumberResult:
    if result.number % 2 != 0:
        raise ValueError("The number must be even.")
    return result

@cf.flow
def check_even_number(number: int):
    # Create a task with a result validator
    result = cf.run(
        objective="Check if the number is even",
        result_type=NumberResult,
        result_validator=validate_even,  # Attach the result validator
        context={"number": number}
    )
    return result

# Example usage
try:
    # Test with an even number
    even_result = check_even_number(4)
    print(f"✅ The number {even_result.number} is even.")

    # Test with an odd number
    odd_result = check_even_number(5)
    print(f"✅ The number {odd_result.number} is even.")
except ValueError as e:
    print(f"❌ Validation failed: {str(e)}")

Output()

✅ The number 4 is even.


Output()

23:21:02.933 | ERROR   | Task run 'Tool call: mark_task_a2bca8e6_successful' - Task run failed with exception: ValueError('The number must be even.') - Retries are exhausted
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 781, in run_context
    yield self
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 1337, in run_task_sync
    engine.call_task_fn(txn)
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 804, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/tools/tools.py", line 61, in run
    result = self.fn(**input)
             ^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/tasks/task.py", line 643, in succeed
    self.mark_successful(result=result_schema(**kwargs))
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/tasks/task.py", line 570, in mark_successful
    self.result = self.validate_result(result)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/tasks/task.py", line 759, in validate_result
    result = self.result_validator(result)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/_b/kmlh75s1007df0jz0xtr03fw0000gn/T/ipykernel_19830/319019531.py", line 8, in validate_even
    raise ValueError("The number must be even.")
ValueError: The number must be even.

23:21:02.937 | ERROR   | Task run 'Tool call: mark_task_a2bca8e6_successful' - Finished in state Failed('Task run encountered an exception ValueError: The number must be even.')

...

23:21:04.943 | ERROR   | Task run 'Run task: Task #a2bca8e6 ("Check if the number is even")' - Finished in state Failed('Task run encountered an exception ValueError: 1 task failed: - Task #a2bca8e6 ("Check if the number is even"): The number 5 is not even, which does not meet the requirement for marking the task as successful.')

23:21:04.945 | ERROR   | Flow run 'polite-orangutan' - Encountered exception during execution: ValueError('1 task failed: - Task #a2bca8e6 ("Check if the number is even"): The number 5 is not even, which does not meet the requirement for marking the task as successful.')
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/flow_engine.py", line 710, in run_context
    yield self
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/flow_engine.py", line 754, in run_flow_sync
    engine.call_flow_fn()
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/flow_engine.py", line 733, in call_flow_fn
    result = call_with_parameters(self.flow.fn, self.parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/decorators.py", line 113, in wrapper
    return fn(*wrapper_args, **wrapper_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/_b/kmlh75s1007df0jz0xtr03fw0000gn/T/ipykernel_19830/319019531.py", line 14, in check_even_number
    result = cf.run(
             ^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/run.py", line 128, in run
    results = run_tasks(
              ^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/tasks.py", line 1002, in __call__
    return run_task(
           ^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 1526, in run_task
    return run_task_sync(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 1339, in run_task_sync
    return engine.state if return_type == "state" else engine.result()
                                                       ^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 482, in result
    raise self._raised
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 781, in run_context
    yield self
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 1337, in run_task_sync
    engine.call_task_fn(txn)
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/task_engine.py", line 804, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/default/lib/python3.11/site-packages/controlflow/run.py", line 62, in run_tasks
    raise ValueError(
ValueError: 1 task failed: - Task #a2bca8e6 ("Check if the number is even"): The number 5 is not even, which does not meet the requirement for marking the task as successful.

23:21:04.970 | ERROR   | Flow run 'polite-orangutan' - Finished in state Failed('Flow run encountered an exception: ValueError: 1 task failed: - Task #a2bca8e6 ("Check if the number is even"): The number 5 is not even, which does not meet the requirement for marking the task as successful.')

❌ Validation failed: 1 task failed: - Task #a2bca8e6 ("Check if the number is even"): The number 5 is not even, which does not meet the requirement for marking the task as successful.


05:15:17.660 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 907.78611 seconds to run, which is longer than its loop interval of 5 seconds.
05:15:17.668 | WARNING | prefect.server.services.failexpiredpauses - FailExpiredPauses took 907.792441 seconds to run, which is longer than its loop interval of 5.0 seconds.
05:15:17.672 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 907.795649 seconds to run, which is longer than its loop interval of 5.0 seconds.
07:15:15.579 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 904.789296 seconds to run, which is longer than its loop interval of 5 seconds.
07:15:15.581 | WARNING | prefect.server.services.failexpiredpauses - FailExpiredPauses took 904.792863 seconds to run, which is longer than its loop interval of 5.0 seconds.
07:15:15.585 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 904.793403 seconds to run, 